<a href="https://colab.research.google.com/github/ctiennot/14nov/blob/main/Bayesian_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to this tutorial! During those two hours we'll have the opportunity to:
- think about spaceships 🚀
- disover Bayesian methods, mainly hierarchical modelling and logistic regression
- use the [Numpyro](https://num.pyro.ai/en/latest/getting_started.html) framework for Bayesian inference in python

Let's get started!

Credit: some parts of this tutorial are shamelessly inspired by [this great tutorial](https://www.stat.ubc.ca/~bouchard/courses/stat520-sp2021-22/Hierarchical_models.html).

<p align="center"><img src="https://www.explainxkcd.com/wiki/images/7/78/frequentists_vs_bayesians.png" width="400"/></p>

# PART 0: Introduction and Setup

## Setup

In [ ]:
pip install numpyro pandas numpy matplotlib arviz

In [ ]:
from arviz import plot_trace
import jax
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
import statsmodels.api as sm

import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

## Motivation: chasing fake news

<p align="center"><img src="https://github.com/ctiennot/bayesian_tutorial/blob/main/pictures/bb_screenshot.jpg?raw=true" width="700"/></p>

In this [2023 BBC article](https://www.bbc.com/future/article/20230518-what-are-the-odds-of-a-successful-space-launch) we can read:

> Three of the Super Heavy booster's 33 engines did not fire on launch, and more failed as it lifted further into the air. Nearly four minutes after initial lift-off, the rocket started tumbling wildly and was deliberately exploded over the Gulf.

Also an empirical estimate of the overall probability of failure is reported:

> That works out to around a 4% failure rate – one in every 25 launches.

Is this true? As rigorous statisticans should we trust this figure without further investigations?

## Bayesian 101: what you need to understand for the tutorial

But first let's go back to the basics of statistics and bayesian inference. We'll try to re-explain it briefly, feel free to skip this part if you already know about all that.

Disclaimer: the topic is broad and it seems pretty hard to understand this stuff in a one morning tutorial (but we'll do our best). If you're really interested consider reading some good books on the matter, for instance:
- [Bayesian Statistics the Fun Way](https://www.amazon.com/Bayesian-Statistics-Fun-Will-Kurt/dp/1593279566/) (I haven't read it, but seems like a gentle introduction)
- [Bayesian Data Analysis by A.Gelman](https://www.amazon.com/Bayesian-Analysis-Chapman-Statistical-Science/dp/1439840954) (the reference in the field, a bit technical but with many concrete and fascinating examples)

We can not really go on without a quick reminder about the bayes theorem:

<p align="center"><img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*Sj8irBMk53oVKweFttSfGA.jpeg" width="700"/></p>

Let's look at each term:
- the <font color='orange'>likelihood</font> can be thought of as "given my model and its parameters what are the odds of observing my data".
- The "belief" for a model is a set of parameters. So the <font color='orange'>posterior</font> says "what's the evidence for my belief / model parameters given the data I observe".
- The <font color='orange'>prior</font> is our initial guess about the evidence for the set of parameters before observing the data.

So the Bayes formula just tells use <font color='orange'>how to rigorously update our prior knowledge</font> about an event (or a set of parameters for a model) given the data we observe.

<p align="center"><img src="https://pbs.twimg.com/media/CE5r1ZMUkAAMWIC.png" width="500"/></p>

A simplified way to put it is that:
- Frequentist folks care about the optimizing the Likelihood: write a model (with some parameters let's say), compute the probability / "likelihood" of the data you observe in your model and find the parameters that give the greatest one.
- in the Bayesian framework we will have a prior belief about the parameters of the model and we will update it according to the data we observe

The way Bayesian does the later is by treating <font color='orange'>**model parameters as random variables**</font>. In comparison, in the frequentist world we consider only pointwise estimates: our estimate is only a single value (we can estimate its uncertainty on the side though, let's not be manicheans).

An example is worth a thousand words: let's assume I flip a coin 10 times and get 9 heads.



- My model is a [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution) with parameter p (the probability of getting "Head")
- We could write the likelihood and get our frequentist estimator $\hat \theta=\frac {n_{heads}}{n} = \frac{9}{10} = 0.9$

Now if I'm doing it in a Bayesian way, I consider that $\theta$ is not fixed, $\theta$ is a random variable:
- I choose a prior distribution for it, let's say a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) (it's a natural choice for a parameter between 0 and 1).
- let's say I choose a Beta($\alpha$, $\beta$) (we'll come back later to the impact of the prior choice)


We can write the equations (take a deep breath, everything will be fine):
- first the likelihood:

$$ p(data|\theta) = \binom {n_{heads} + n_{tails}}{n_{heads}} \theta^{n_{heads}} (1 - \theta)^{n_{tails}}$$

- then the prior distribution (it's the density of a Beta distribution, cf the wikipedia page for instance):

$$p(\theta) = \frac{\theta^{\alpha -1}(1-\theta)^{\beta -1}}{\mathrm{B}(\alpha ,\beta )}$$

($\mathrm{B}(\alpha ,\beta )$ is there for the density to integrate to 1, don't worry too much about it).

- now we combine them using the Bayes formula:
$$ p(\theta|data) = \frac{p(data|\theta) p(\theta)}{p(data)} $$

$$ = \binom {n_{heads} + n_{tails}}{n_{heads}} \frac{\theta^{n_{heads}} (1 - \theta)^{n_{tails}} \theta^{\alpha -1}(1-\theta)^{\beta -1}}{\mathrm{B}(\alpha ,\beta ) p(data)} $$

$$ = \frac{\theta^{n_{heads} + \alpha -1} (1 - \theta)^{n_{tails} + \beta -1 }}{\text{something that doesn't depend on } \theta} $$

$$∝ \theta^{n_{heads} + \alpha -1} (1 - \theta)^{n_{tails} + \beta -1 }$$

The $∝$ symbol means "proportional to" and here the trick is to see that we don't really care about things that are not related to $\theta$. In other words it's already a lot to know the posterior density up to a constant, we can deal with that (using MCMC for instance). For know just trust me and assume that it's enough :)

In this simple example we can simply "recognize" the distribution: it's a $Beta(n_{heads} + \alpha, n_{tails} + \beta)$. So easy! It's a particular case where the prior and the posterior are in the same family (we say they are "conjugate").

The posterior mean is
$$ \frac{n_{heads} + \alpha}{n_{heads} + n_{tails} + \beta + \alpha}$$

($\alpha$ and $\beta$ are in some sense equivalent to some pseudo data points: $\alpha$ extra heads and $\beta$ extra tails).

Now let's say I chose $\alpha = 1$ and $\beta = 1$ to set my prior (it's a uniform distribution actually, just check). I can now plot my posterior distribution:

In [ ]:
from scipy.stats import beta, binom

alpha_ =  1
beta_ = 1
n_heads, n_tails = 9, 1
likelihood = lambda theta: binom.pmf(n_heads, n_heads + n_tails, theta)

x = np.arange (0.01, 1, 0.01)

f, ax = plt.subplots(1, 1, figsize=(15, 4))
plt.plot(x, beta.pdf(x, alpha_, beta_), label="Prior density")
plt.plot(x, likelihood(x), label="likelihood")
plt.plot(x, beta.pdf(x, n_heads + alpha_, n_tails + beta_), label="Posterior density")
plt.title("Coin flip example: uniform prior")
plt.xlabel("theta")
plt.axvline(x=n_heads / (n_heads + n_tails), label="Frequentist estimate", linestyle="--", color="orange")
plt.axvline(x=(n_heads + alpha_) / (n_heads + n_tails + alpha_ + beta_), label="Posterior mean", linestyle="--", color="green")

plt.legend()
plt.show()

If we look at the posterior: taking the mode (peak) of the posterior as an estimator raises the same result as the maximum likelihood / frequentist estimate but taking the posterior mean we are shifting a bit to the left: <font color='orange'>the prior regularizes a bit the estimate towards 0.5</font>. But the interest of Bayesian inference is not to get back to single point wise estimates but to <font color='orange'>embrace the full posterior distribution</font>: look at its shape and take a moment to realize we went from a single vertical line to this nice curve.

But let's be more opiniated: what if we have strong prior knowledge about the coin being a fair coin. We can update our prior to reflect that: we need a prior that puts more weight at the parameter values we think are more likely (here 0.5):

In [ ]:
alpha_ =  30
beta_ = 30
n_heads, n_tails = 9, 1
likelihood = lambda theta: binom.pmf(n_heads, n_heads + n_tails, theta)

x = np.arange (0.01, 1, 0.01)

f, ax = plt.subplots(1, 1, figsize=(15, 4))
plt.plot(x, beta.pdf(x, alpha_, beta_), label="Prior density")
plt.plot(x, likelihood(x), label="likelihood")
plt.plot(x, beta.pdf(x, n_heads + alpha_, n_tails + beta_), label="Posterior density")
plt.title("Coin flip example: informative prior")
plt.xlabel("theta")
plt.axvline(x=n_heads / (n_heads + n_tails), label="Frequentist estimate", linestyle="--", color="orange")
plt.axvline(x=(n_heads + alpha_) / (n_heads + n_tails + alpha_ + beta_), label="Posterior mean", linestyle="--", color="green")
plt.legend()
plt.show()

And we see the posterior distribution is impacted a lot because of our (very informative) prior distribution.

# PART I: What's the probability of a successul rocket launch?

Let's now apply all that to our initial question: rocket launches 🚀

## A simple case: 3 rocket launches

 We will consider predicting the success/failure for the next launch of the **Delta 7925H rocket**, which as of November 2020 has been **launched 3 times**, with 0 failed launches. Maximum likelihood would therefore give an estimate of 0% for the chance of failure. Obviously we will not use this crazy estimate today and instead use Bayesian methods.

Let us start with the simplest Bayesian model for this task: we assume the **three launches are independent**, biased coin flips, all with a shared probability of failure (bias) given by an unknown parameter failureProbability. In other words, we assume a binomial likelihood. We also assume a beta prior on the failureProbability, the parameter p of the binomial (the parameter n is known and set to the number of observed launches, numberOfLaunches).


We will now code this in Numpyro to carry the estimation:

In [ ]:
def model(n_launches: int, n_failures: int):
    failure_probability = numpyro.sample("failure_probability", dist.Beta(1, 1))
    return numpyro.sample("failures", dist.Binomial(n_launches, failure_probability), obs=n_failures)

In [ ]:
mcmc = MCMC(NUTS(model), num_warmup=500, num_samples=3000)
mcmc.run(jax.random.PRNGKey(0), n_launches=3, n_failures=0)
posterior_samples = mcmc.get_samples()

Some explanations about the above code:
- in Numpyro we defined a "model" function that takes as input some observed data (n_launches and n_failures)
- inside the function we write the probabilistic model: random variables are declared with `numpyro.sample(name, distribution)` where we specify its name and the distribution it follows.
- if we observe something, we add the `obs=` parameter to specify which data were observed for this variable and we condition the posterior probability on that.

The MCMC block conducts the inference: we won't explain it in details here but the overall idea is that since the model might be complex, we might not be able to compute the posterior distribution analytically so we need to approximate it using this MCMC scheme: it builds a Markov Chain of values that should converge to an equilibrium state where our posterior distribution can be approximated by samples from the chain.

In practice: Numpyro handles that for us and <font color='orange'>we just care about the posterior samples we retrieve: those are our results</font> (for real one should check the chains are mixing well etc. but this is only a tutorial...).

In [ ]:
posterior_samples.keys()

We can look at the estimated parameter:

In [ ]:
mcmc.print_summary()

We see that our confidence interval is [0, 0.43] for the failure rate and a median estimate would be 16% chances of failure.

Let's also look at the posterior samples (as an histogram on the left, as the MCMC chain on the right):

In [ ]:
plot_trace(mcmc)
plt.show()

The distribution has more mass near 0 but it's pretty wide: this reflects our uncertainty about the estimate because we have only 3 observations.

## The main objection: prior sensitivity

To motivate the need for hierarchical models, we will now use our rocket example to demonstrate that the **posterior distribution is sensitive to the choice of prior when the dataset is small**.

But before doing this, we will transform the parameters of the beta prior ($\alpha$ and $\beta$) into a new pair of parameters ($\mu$ and $s$) which are more interpretable. The transformation we use is:
$$ \mu = \frac{\alpha}{\alpha + \beta} $$ and $$s = \alpha + \beta$$

where $\mu \in (0, 1)$ and $s > 0$, which is invertible with an inverse given by
$$\alpha = \mu s$$ and $$\beta = (1 - \mu) s$$

The interpretation of this reparameterization is:

$\mu$: the mean of the prior distribution,

$s$: a measure of "peakiness" of the prior density; higher $s$ correspond to a more peaked density; roughly, $s$ ≈ number of data point ("pseudo-observations") that would make the posterior peaked like that.

Here is the reparameterized code, which gives the same approximation as before (
$\mu$ is labelled mu in the code):

In [ ]:
def model_reparameterized(n_launches: int, n_failures: int):
    mu = 0.5
    s = 2
    failure_probability = numpyro.sample("failure_probability", dist.Beta(mu * s, (1 - mu) * s))
    return numpyro.sample("failures", dist.Binomial(n_launches, failure_probability), obs=n_failures)

In [ ]:
mcmc = MCMC(NUTS(model_reparameterized), num_warmup=500, num_samples=3000)
mcmc.run(jax.random.PRNGKey(0), n_launches=3, n_failures=0)
posterior_samples = mcmc.get_samples()

In [ ]:
mcmc.print_summary()

In [ ]:
plot_trace(mcmc)
plt.show()



---
🖊 EXERCISE 1

Change the prior mean parameter, e.g. from the "neutral" value of
0.5 to a more "optimistic" value of 0.1, to demonstrate sensitivity of the posterior to the choice of prior. Report the change in the credible intervals, which should shrink considerably when moving to the optimistic prior.

---



## Side data

How to attenuate prior sensitivity? The key idea that will lead us to hierarchical models is to **use side data to inform the prior**. Back to the rocket application, recall that we are interested in one type of rocket called Delta 7925H. In this context, side data takes the form of success/fail launch data from other types of rockets.

Here is the data that we will use (showing only the first 10 rows out of 334 rows):

In [ ]:
path = "https://raw.githubusercontent.com/ctiennot/bayesian_tutorial/main/data/counts.csv"
df_launches = pd.read_csv(path, index_col=0)
df_launches.head(10)

## Non Bayesian use of the data

Before we describe the Bayesian way to use side data, let us briefly go over a more naive method. For now, let us focus on the prior parameter $\mu$.

Here is the naive method to determine the prior parameter $\mu$ from our "side data":

Estimate the failure probability $p_i$ for each rocket type $i$ (e.g. using maximum likelihood estimation, for binomial this is just the fraction of rocket failures for each rocket type). Let us denote each estimate by $\hat p_i$

Fit a distribution over all the estimated failure probabilities
$\hat p_i$'s

Use this fit distribution as the prior on $\mu$.

In [ ]:
estimated_p_i_s = df_launches["numberOfFailures"] / df_launches["numberOfLaunches"]

plt.figure(figsize=(15, 4))
plt.hist(estimated_p_i_s, bins=30)
plt.xlabel("$\hat p_i$ (estimated failure rate per rocket types)")
plt.ylabel("Count")
plt.title("The ugly way to build a prior distribution")
plt.show()


This method provides some intuition on how we can use side data to inform our choice of prior distribution. However this naive method has some weaknesses. A first weakness is that it is less obvious how to proceed with the precision parameter $s$. A second weakness is that the distribution estimated in step 2 above has some irregularities, visible in the histogram of MLE-estimated
$\hat p_i$'s in our dataset

---
🖊 EXERCISE 2

Notice the bumps at 1/2 and 1 in the above histogram. Speculate on the cause of these two bumps or irregularities in the histogram.

---

## Bayesian use of side data: the Hierarchical Model

<p align="center"><img src="https://github.com/ctiennot/bayesian_tutorial/blob/main/pictures/hierarchical-gm.jpg?raw=true" width="700"/></p>

A general principle in Bayesian inference is to treat **unknown quantities as random**. This is also the essence of a Bayesian hierarchical model: let us treat $\mu$ and $s$ as random and let the side data inform their posterior distribution.

To visualize what a hierarchical model is, let us look at the corresponding graphical model, shown here. The circles are random variables, the squares, non-random. Shaded nodes are observed, white ones, unknown. An arrow from a variable $X$ to $Y$ means that the generative model for $Y$ has a parameter that depends on $X$. The rectangle enclosing the bottom three nodes is called a plate, encoding the fact that the "plated" random variables (those inside the rectangle) are copied, once for each rocket type (i.e. for each row in the data table shown earlier).

The model inside the plate is just several copies of the BasicReparameterized model. You can think of this as the "bottom level" of a hierarchy. What we have added in the hiearchical model is the "top level", which consists of two random variables, one for $\mu$, and one for $s$, each with its own prior distribution. Here we will use a beta prior for $\mu$ and an exponential prior for $s$.

Let's implement this in Numpyro:

In [ ]:
def hierarchical_model_slow(n_launches: List[int], n_failures: List[int], rocket_type_names: List[str]):
    assert len(n_launches) == len(n_failures) == len(rocket_type_names)
    n_rocket_types = len(rocket_type_names)

    # prior distribution parameters
    mu_mean, mu_s = 0.5, 2

    # prior distributions for the common failure probabilities distribution
    mu = numpyro.sample("mu", dist.Beta(mu_mean * mu_s, (1 - mu_mean) * mu_s)) # mu has become a random variable!
    s = numpyro.sample("s", dist.Exponential(0.1)) # s has become a random variable!

    for i in range(n_rocket_types):  # it would be more efficient to vectorize here but it's easier to read for the tutorial...
      rocket_type = rocket_type_names[i]
      failure_probability = numpyro.sample(f"failure_probability_{rocket_type}", dist.Beta(mu, s)) # a failure probability for this rocket type is sampled from the common distribution of failure probabilities
      return numpyro.sample(f"failures_{rocket_type}", dist.Binomial(n_launches[i], failure_probability), obs=n_failures[i]) # we observe the failures for this rocket type

Turns out the for loop is super slow in practice and we will vectorize things to go faster :)

In [ ]:
def hierarchical_model(n_launches: List[int], n_failures: List[int], rocket_type_names: List[str]):
    assert len(n_launches) == len(n_failures) == len(rocket_type_names)
    n_rocket_types = len(rocket_type_names)

    # prior distribution parameters
    mu_mean, mu_s = 0.5, 2

    # prior distributions for the common failure probabilities distribution
    mu = numpyro.sample("mu", dist.Beta(mu_mean * mu_s, (1 - mu_mean) * mu_s)) # mu has become a random variable!
    s = numpyro.sample("s", dist.Exponential(0.1)) # s has become a random variable!

    # Numpyro can vectorize operations using this plate mechanism :)
    with numpyro.plate("rocket_types", n_rocket_types):
      failure_probabilities = numpyro.sample(f"failure_probabilities", dist.Beta(mu * s, (1 - mu) * s))
      return numpyro.sample(f"failures", dist.Binomial(n_launches, failure_probabilities), obs=n_failures)

Some explanations of the above code:

- the "plate" statement correspond to the to the rectangle on the graph above.
- `mean_mu` and `mu_s` are needed to parametrize the $\mu$ distribution
- the $s$ distribution is also parametrize using 0.1 in the exponential


Now we'll run the MCMC chain:

In [ ]:
mcmc = MCMC(NUTS(hierarchical_model), num_warmup=500, num_samples=3000)
mcmc.run(
    jax.random.PRNGKey(0),
    n_launches=df_launches["numberOfLaunches"].values,
    n_failures=df_launches["numberOfFailures"].values,
    rocket_type_names=df_launches["rocketType"].values
  )
posterior_samples = mcmc.get_samples()

In [ ]:
mcmc.print_summary()

In [ ]:
# can take several minutes to plot!
plot_trace(mcmc)
plt.tight_layout()
plt.show()

We now have a lot of parameters in the MCMC summary 😮 Because of the plate statement we have one node / random variable per failure probability, so per rocket type (and we have 335 of them). But look at the $\mu$ estimate: we now have a 0.12 median and [0.10, 0.13] as our (90%) credible interval.

Note that the new model still has prior distributions at the top level, and hence parameters to set for these new top level priors. Are we back to square one? No, as **it turns out these top-level parameters will be generally less sensitive** as you will verify in the following exercise.



---
🖊 EXERCISE 3

Change the prior mean on the $\mu$ variable, e.g. from the "neutral" value of 0.5 to a more "optimistic" value of 0.1, to demonstrate decreased sensitivity of the posterior to the prior choice in the hierarchical model compared to the basic model. Report the change in the credible intervals for Delta 7925H's failure probability, which should be almost the same when moving to an optimistic prior.

---



## Understanding what just happened

How to explain this reduced sensitivity? One heuristic is based on the graphical model. For each unobserved (white) node in the graphical model, the number of edges connected to that node is often indicative of how peaky the posterior distribution is, and peaky posteriors tend to be less sensitive to prior choice.

Consider for example the graphical model for the basic model (left below). Note that the variable $p$ (failure probability for the Delta 7925H) has only 3 edges connected to it. Even if we expanded the observed number of failures (F) to a list of Bernoulli random variables, this would only lead to two more connections. In contrast, with the hierarchical model (right), the top nodes have 334 connections (the number of rocket types, i.e. rows in the data table). The theorem behind this vague heuristic is the [Bernstein–von Mises theorem](https://en.wikipedia.org/wiki/Bernstein%E2%80%93von_Mises_theorem).

<p align="center">
<img src="https://www.stat.ubc.ca/~bouchard/courses/stat520-sp2021-22/figs/hierarchical-gm-before.jpg" height="300"/>
<img src="https://www.stat.ubc.ca/~bouchard/courses/stat520-sp2021-22/figs/hierarchical-gm-after.jpg" height="300"/>
</p>

## Getting back to the BBC article

So is the claim in the BBC article false? We did find a 10% to 13% to be a 90% credible interval for our "global" failure rate which doesn't include the 5% figure stated in the article.

Well let's first state a few things:
- our dataset stops at Nov 2020
- the data go back up to 1957 and it's clear that there have been improvement and probably the failure rate has decreased in time
- all launchers are not equal (hence the groups in our Bayesian model)

The article states:
>"Last year (2022), there were 186 launches," he says. These carried 2,509 satellites into orbit, the majority of those being Starlink, SpaceX's satellite internet constellation. "

If we take the Starlink launches, most of those are using the Falcon 9  launch vehicle.

<p align="center">
<img src="https://github.com/ctiennot/bayesian_tutorial/blob/main/pictures/falcon9_screenshot.jpg?raw=true" height="300"/>
</p>

In [ ]:
df_launches.loc[df_launches["rocketType"].str.contains("Falcon")]

In [ ]:
falcon9_i = list(df_launches["rocketType"].values).index("Falcon 9")
falcon9_i

In [ ]:
falcon_9_post_samples = posterior_samples["failure_probabilities"][:, falcon9_i]
median = np.mean(falcon_9_post_samples)
ci = np.percentile(falcon_9_post_samples, (2.5, 97.5))

plt.figure(figsize=(15, 4))
plt.hist(falcon_9_post_samples, bins=50)
plt.title("Flacon 9 failure rate estimate")
plt.xlabel("Failure rate")
plt.axvline(median, linestyle="--", color="red", label="Median")
plt.text(median + 0.001, 200, f"Median = {median:.2%}", color="red")
plt.axvspan(*ci, facecolor='red', alpha=0.1, label="95% CI")
plt.legend()
plt.show()

So the BBC estimate might not be wrong, but it's vague regarding what type of rockets and we can blame them for not giving confidence interval. Bad student.

---
If you got to this part of the tutorial, good game, that's already great! I initially planned to stop there (the original tutorial did) but then I thought "what if some folks finish too early"?

<p align="center">
<img src="https://www.shutterstock.com/image-photo/funny-cocktail-dog-holding-martini-260nw-132475163.jpg" height="300"/>
</p>

Feel free to stop there if you're bored or go have a quick coffe and/or chat with your colleagues before keeping the hard work.

# PART II: More fact checking, more models, more fun

## Is Wade right? A good old logistic regression to the rescue

We've also read in the BBC article:
>"Typically, first or second launch, you expect something like 30% of them to fail," says Wade. "Then things start to get better thereafter, by the time you're up to the 10th flight, you're probably looking at a less than 5% failure rate."

And we want to double check what Wade is saying. Let's load some more detailed data (one row per launch instead of agregated figures):

In [ ]:
df_detailed = pd.read_csv("https://raw.githubusercontent.com/ctiennot/bayesian_tutorial/main/data/processed.csv", index_col=0)
df_detailed.head(10)

We can plot a few descriptive statistics, for instance the number of launches through time:

In [ ]:
plt.figure(figsize=(15, 4))
df_detailed["year"].value_counts().sort_index().plot.bar()
plt.title("Launches through time")
plt.xlabel("Year")
plt.show()

Or the counts per rocket types:

In [ ]:
plt.figure(figsize=(15, 6))
df_detailed["rocketType"].value_counts().head(20).iloc[::-1].plot.barh()
plt.title("Top Rocket types")
plt.xlabel("Number of launches")
plt.grid()
plt.show()

The "launchIndex" variable tells us the index of the launch for the given
"rocketType". So `launchIndex = 1` means it's the first time this rocket type was tried. So could we simply groupby this field and check the failure rate?

In [ ]:
df_detailed["first_rocket_type_launch"] = df_detailed["launchIndex"] == 1

We'll report the Confidence Intervals (CI) in a frequentist way (using a normal approx).

In [ ]:
def compute_bernouilli_CI_width(means, counts):
  # normal approx, centered on means
  return 1.96 * np.sqrt(means * (1 - means) / counts)

success_per_launch_index = df_detailed.groupby("launchIndex").agg({'success':['mean', 'std', 'count']})
success_per_launch_index["ci_width"] = compute_bernouilli_CI_width(success_per_launch_index[('success',  'mean')], success_per_launch_index[('success',  'count')])
success_per_launch_index.head(5)

Let's plot those by the launch index to see what it gives:

In [ ]:
n_to_display = 20
plt.figure(figsize=(15, 4))
success_per_launch_index.head(n_to_display)[('success',  'mean')].plot.bar(yerr=success_per_launch_index["ci_width"].head(n_to_display))
plt.xlabel("Launch Index")
plt.ylabel("Estimated success rate")
plt.title("Is the first Launch that unsuccessful?")
plt.ylim((0.6, 1))
plt.grid()
plt.axhline(y=0.7, color="red", linestyle="--")
t = plt.text(0, 0.72, "Wade's first launch success rate estimate", color="red")
t.set_bbox(dict(facecolor='white', alpha=0.6, edgecolor="white"))
plt.show()

This (frequentist) simple analysis would prove Wade estimate to be wrong: the success rate would be more around 83% for first launches (i.e. 17% failure rate, not 30%).

But aren't there any bias here? The success might also depend of the year (systems reliability might improve) of the country, of the rocketType (as in first part) etc. We should account for those.

A natural way to do so is by **fitting a logistic regression to predict the success / failure of a rocket launch**. That way if we put the right variables in there we will be able to control for those biases. We could use the sklearn one but it won't report confidence intervals and other "stats" things so we'll use the statsmodel one.

In [ ]:
X = (pd.get_dummies(df_detailed[["year", "first_rocket_type_launch", "country"]]) * 1).astype(float)
X["year_after_1956"] = X["year"] - 1956
X.drop(columns=["year"], inplace=True)
X.drop(columns=["country_USA"], inplace=True) # USA will be our reference country
y = df_detailed["success"]

In [ ]:
X.head()

In [ ]:
logit_mod = sm.Logit(y, sm.add_constant(X))

In [ ]:
logit_res = logit_mod.fit()

In [ ]:
print(logit_res.summary())

Before jumping to conclusions, let's check if the fit is not too poor. For linear regression we would check the residuals for instance to see if the model fit weel the data. Here for a logistic regression a common way is to split the predicted probabilities in bins / buckets and see if the empirical output rate matches the predicted one on the bucket. This is called **checking "model calibration"**.

In [ ]:
import numpy as np
from sklearn.calibration import calibration_curve

def plot_calibration_curve(logit_res, X, y):
  prob_true, prob_pred = calibration_curve(y, logit_res.predict(sm.add_constant(X)), n_bins=10, strategy="quantile")

  f = plt.figure(figsize=(10, 4))
  plt.plot(prob_pred, prob_true, "o-")
  plt.plot([0.7, 1], [0.7, 1], linestyle="--")
  plt.xlabel("Predicted proba")
  plt.ylabel("Real proba in bucket")
  plt.title("Calibration plot for the Logistic regression")
  plt.show()

plot_calibration_curve(logit_res, X, y)

We'll see we can do better but let's keep up for now.

Parameters are living in the logit space and are not super interpretable but we can compute <font color='orange'>marginal effects</font> of those: put simply "how does a unit increase of the variable impact the output variable".

In [ ]:
print(logit_res.get_margeff().summary())

- a first launch decreases by <font color='orange'>6 points of percentage</font> the success probability of a successful launch.
- in average a China's rocket launch success probability is decreased by 6.2 percent points compared to a USA one. Europe and Russia effects are not significant.
- in average each year that past since 1957 increased by 0.28 points of percentage the success probability of launching rockets.

But that's a marginal effect, what about concrete probabilities? Let's See for a US rocket through the years what it gives:

In [ ]:
fitted_params = logit_res.params
for year in [1960, 1970, 1980, 1990, 2000, 2010, 2020]:
  not_first_time_estimate = 1 / (1 + np.exp(-(fitted_params["const"] + fitted_params["year_after_1956"] * (year - 1956))))
  first_time_estimate = 1 / (1 + np.exp(-(fitted_params["const"] + fitted_params["first_rocket_type_launch"] + fitted_params["year_after_1956"] * (year - 1956))))
  print(f"Success for a first time US launch in {year}: {first_time_estimate:.2%}")
  print(f"Success for a non first time US launch in {year}: {not_first_time_estimate:.2%}\n")

The effect of the year seems huge. Also having a linear dependency on it is a bit weird, let's check what the data tell us:

In [ ]:
df_detailed.groupby("year")["success"].mean().apply(lambda p: np.log(p / (1 - p))).plot()
plt.title("Avg success rate accross year (logit scale)")
plt.ylabel("Logit(empirale rate)")
plt.show()

We see that a linear coefficient on year doesn't really make sense, why not add a coefficient on the log of the year?

In [ ]:
X["log_year_after_1956"] = np.log(X["year_after_1956"])
logit_mod = sm.Logit(y, sm.add_constant(X))
logit_res = logit_mod.fit()
print(logit_res.summary())
plot_calibration_curve(logit_res, X, y)

The added coefficient is statistically significant and the calibration curve is way better. Let's check again our probabilities:

In [ ]:
fitted_params = logit_res.params
for year in [1960, 1970, 1980, 1990, 2000, 2010, 2020]:
  common_logit_effect = fitted_params["const"] + fitted_params["year_after_1956"] * (year - 1956) + fitted_params["log_year_after_1956"] * np.log(year - 1956)
  not_first_time_estimate = 1 / (1 + np.exp(-common_logit_effect))
  first_time_estimate = 1 / (1 + np.exp(-(common_logit_effect + fitted_params["first_rocket_type_launch"])))
  print(f"Success for a first time US launch in {year}: {first_time_estimate:.2%}")
  print(f"Success for a non first time US launch in {year}: {not_first_time_estimate:.2%}\n")

Interesting... There could be many more things to do there, but I'm already taking too much time to prepare this tutorial and I want to get to the Bayesian counterpart of the logistic regression so I'll leave you carry further analysis and improve the model if you want. What we can say is:
> <font color='orange'>Given our data, and if we trust this basic model it seems a 30% failure rate for a (US) first launch is exagerated</font>.

Let's be cautious because we don't have the recent year data and our model is not to be trusted blindly, we didn't carry in depth goodness of fit checks (execpt a quick calibration curve one).



---
🖊 EXERCISE 4 (ideas)

Try to improve this quick model / analysis:
- what features could you add?
- Can you spot an issue with the way we dealt with the "year" variable?
- would it be a good idea to split our data in a train / test to assess the predictive power of our logistic regression? If so how would you split? What kind of AUC / f1 score do you except to get?
- Could you think of something else than logistic regression? Maybe trees? What are the pros and cons of this?
- We have many boolean variables in our logistic regression, what could we do about it? At what price does it come?

And please give a shot at any other idea you might have.

---



## Bayesian logistic regression

Now notice that in the above **we only got point estimates** of our probabilities like "for a US rocket in 1990 launching for the first time the probability is 93.68%" but **what if we want a confidence inteval on this**? The logistic regression outputs give us CI on coefficients but not on the predictions. This is not impossible to derive (see [this post](https://stats.stackexchange.com/questions/354098/calculating-confidence-intervals-for-a-logistic-regression) for instance) in the frequentist world but this looks tricky to me (feel free to disagree) for something so simple in the Bayesian world (at the cost of more compute power to be fair). And the Bayesian method comes with many other advantages, stay tuned.

So let's see how we can fit the same logistic regression in a Bayesian way.


In [ ]:
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import numpy as np
import jax.numpy as jnp
from jax import random


def model(years, country_idx, is_first_launch, y=None):
    # Prior for the intercept
    intercept = numpyro.sample('intercept', dist.Normal(0, 20))

    # Priors for the coefficients of years and log(years)
    beta_years = numpyro.sample('beta_years', dist.Normal(0, 5))
    beta_log_years = numpyro.sample('beta_log_years', dist.Normal(0, 5))

    # Handling the categorical variable 'country_idx'
    num_countries = len(np.unique(country_idx))
    beta_country = numpyro.sample('beta_country', dist.Normal(jnp.zeros(num_countries), jnp.ones(num_countries)))

    beta_first_launch = numpyro.sample('beta_first_launch', dist.Normal(0, 1))

    # Linear combination of inputs (including log(years))
    log_years = jnp.log(years)
    logits = numpyro.deterministic(
        "logits",
        intercept + beta_years * years + beta_log_years * log_years + beta_country[country_idx] + is_first_launch * beta_first_launch
    )
    probas = numpyro.deterministic("probas", jax.scipy.special.expit(logits))

    with numpyro.plate('data', len(years)):
        obs = numpyro.sample('obs', dist.Bernoulli(logits=logits), obs=y)


def run_inference(model, years, country_idx, is_first_launch, y, rng_key, num_warmup=500, num_samples=1000):
    # Running MCMC
    kernel = NUTS(model)
    mcmc = MCMC(kernel, num_warmup=num_warmup, num_samples=num_samples)
    mcmc.run(rng_key, years, country_idx, is_first_launch, y)
    return mcmc, mcmc.get_samples()



countries = df_detailed["country"].values
# Convert categorical variables to integers
_, country_idx = np.unique(countries, return_inverse=True)
years = df_detailed["year"].values - 1956
is_first_launch = df_detailed["first_rocket_type_launch"].values
y = df_detailed["success"].values

# Run the model
rng_key = random.PRNGKey(0)
mcmc, samples = run_inference(model, years, country_idx, is_first_launch, y, rng_key)

# Print summary or analyze samples...

We just used simple normal priors on each coefficient, the model remains the same as before.

In [ ]:
mcmc.print_summary()

In [ ]:
plot_trace(mcmc, var_names=["intercept", "beta_years", "beta_first_launch", "beta_country"])
plt.tight_layout()
plt.show()

The coefficients look similar to those of the former logistic, but now we've got some posterior distributions on those.

And to derive our credible intervals about each prediction, let's introduce a very important concept in Bayesian inference: [**the posterior predictive distribution**](https://en.wikipedia.org/wiki/Posterior_predictive_distribution).

In a nutshell the idea is that the uncertainty about a prediction is coming from two things:
- the **model itself** (in a linear regression setting for instance, we assume some random noise in the model)
- the **uncertainty on the parameter estimates**: because we are not sure about the parameters, we need to propagate this uncertainty to our predictions which are using those estimates.

Or as Wikipedia states it:

>It may seem tempting to plug in a single best estimate but this ignores uncertainty about it and because a source of uncertainty is ignored, the predictive distribution will be too narrow. Put another way, predictions of extreme values will have a lower probability than if the uncertainty in the parameters as given by their posterior distribution is accounted for.

I don't want to insist on the math side, you can wander around and find explanations on the topic if you're interested.

In Numpyro we have an helper "Predictive" function that does teh job: we only need to specify it the model and give it samples from our posterior distribution.

In [ ]:
from numpyro.infer import Predictive
predictive = Predictive(model, posterior_samples={k: v for k, v in samples.items() if k not in ['logits', 'probas']})
predictions_training_set = predictive(jax.random.PRNGKey(10), years=years, country_idx=country_idx, is_first_launch=is_first_launch, y=y)

Let's sanity check the calibration:

In [ ]:
import numpy as np
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y, predictions_training_set["probas"].mean(axis=0), n_bins=10, strategy="quantile")


f = plt.figure(figsize=(10, 4))
plt.plot(prob_pred, prob_true, "o-")
plt.plot([0.7, 1], [0.7, 1], linestyle="--")
plt.xlabel("Predicted proba")
plt.ylabel("Real proba in bucket")
plt.title("Calibration plot for the Bayesian Logistic regression")
plt.show()

Let's now plot a random sample of rocket launches with their CI displayed:

In [ ]:
from numpyro.diagnostics import hpdi

np.random.seed(1804)
sample_obs = np.random.choice(range(len(y)), 50)
pred_mean = predictions_training_set["probas"].mean(axis=0)[sample_obs]
pred_hpdi = hpdi(predictions_training_set["probas"], 0.95)[:, sample_obs]

order = np.argsort(pred_mean)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

plt.errorbar(
    pred_mean,
    range(len(sample_obs)),
    xerr=pred_hpdi[1, :] - pred_mean,
    marker="o",
    ms=5,
    mew=4,
    ls="none",
    alpha=0.8,
)
plt.xlabel("Estimated Success Probability")
plt.ylabel("Some observations sampled at random")
plt.title("Some random 95% credible intervals on predicted values")
plt.show()

And what about a US rocket in 2022?

In [ ]:
us_index = np.unique(country_idx[countries == "USA"])[0]

In [ ]:
predictions_us_2022_first_launch = predictive(jax.random.PRNGKey(10), years=np.array([2022 - 1956]), country_idx=np.array([us_index]), is_first_launch=np.array([True]), y=None)

In [ ]:
plt.figure(figsize=(15, 4))
plt.hist(predictions_us_2022_first_launch["probas"].flatten(), bins=30)
plt.title("Estimated Success Probability for a US Rocket in 2022")
plt.xlabel("Probability of success")
plt.show()

Can we check the evolution through time?

In [ ]:
years_ = range(1960, 2022)
predictions_us_through_time_first_launch = predictive(jax.random.PRNGKey(10), years=np.array([year - 1956 for year in years_]), country_idx=np.array([us_index]), is_first_launch=np.array([True]), y=None)["probas"]
predictions_us_through_time_not_first_launch = predictive(jax.random.PRNGKey(10), years=np.array([year - 1956 for year in years_]), country_idx=np.array([us_index]), is_first_launch=np.array([False]), y=None)["probas"]

In [ ]:
plt.figure(figsize=(15, 4))
plt.plot(years_, predictions_us_through_time_first_launch.mean(axis=0), label="At first launch")
plt.fill_between(
  years_,
  np.percentile(predictions_us_through_time_first_launch, 5, axis=0),
  np.percentile(predictions_us_through_time_first_launch, 95, axis=0),
  alpha=0.2
)
plt.plot(years_, predictions_us_through_time_not_first_launch.mean(axis=0), label="After first launch")
plt.fill_between(
  years_,
  np.percentile(predictions_us_through_time_not_first_launch, 5, axis=0),
  np.percentile(predictions_us_through_time_not_first_launch, 95, axis=0),
  alpha=0.2
)
plt.title("Estimated of a Successful Launch Probability for a US Rocket")
plt.ylabel("Probability of success")
plt.xlabel("Year")
plt.grid()
plt.legend(loc=4)
plt.show()



---
🖊 EXERCISE 5

You are now a Bayesian master, carve your own path.

---



<p align="center">
<img src="https://i.redd.it/hwxab23r2r291.jpg" height="300"/>
</p>